In [1]:
import checklist
from checklist.test_suite import TestSuite
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
from transformers import pipeline

## Sentiment analysis

In [3]:
model = pipeline("sentiment-analysis", device=0)

Add some batching, I run out of memory otherwise:

In [4]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def batch_predict(model, data, batch_size=128):
    ret = []
    for d in chunks(data, batch_size):
        ret.extend(model(d))
    return ret
    

The tests in the paper are for 3-label sentiment analysis (negative, positive, neutral). We adapt this binary model by making everything in the [0.33, 0.66] range be predicted as neutral:

In [5]:
import numpy as np
def pred_and_conf(data):
    # change format to softmax, make everything in [0.33, 0.66] range be predicted as neutral
    preds = batch_predict(model, data)
    pr = np.array([x['score'] if x['label'] == 'POSITIVE' else 1 - x['score'] for x in preds])
    pp = np.zeros((pr.shape[0], 3))
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
    

In [6]:
suite_path = '../release_data/sentiment/sentiment_suite.pkl'
suite = TestSuite.from_file(suite_path)

In [7]:
suite.run(pred_and_conf, n=500)

Running single positive words
Predicting 34 examples
Running single negative words
Predicting 35 examples
Running single neutral words
Predicting 13 examples
Running Sentiment-laden words in context
Predicting 500 examples
Running neutral words in context
Predicting 500 examples
Running intensifiers
Predicting 1000 examples
Running reducers
Predicting 1000 examples
Running change neutral words with BERT
Predicting 5046 examples
Running add positive phrases
Predicting 5500 examples
Running add negative phrases
Predicting 5500 examples
Running add random urls and handles
Predicting 11000 examples
Running punctuation
Predicting 1170 examples
Running typos
Predicting 1000 examples
Running 2 typos
Predicting 1000 examples
Running contractions
Predicting 1038 examples
Running change names
Predicting 3641 examples
Running change locations
Predicting 5500 examples
Running change numbers
Predicting 5500 examples
Running used to, but now
Predicting 500 examples
Running "used to" should reduce
Pr

In [8]:
# suite.visual_summary_table()

In [9]:
suite.summary()

Vocabulary

single positive words
Test cases:      34
Fails (rate):    0 (0.0%)


single negative words
Test cases:      35
Fails (rate):    1 (2.9%)

Example fails:
0.3 0.0 0.7 average
----


single neutral words
Test cases:      13
Fails (rate):    13 (100.0%)

Example fails:
1.0 0.0 0.0 commercial
----
0.0 0.0 1.0 find
----
0.0 0.0 1.0 Israeli
----


Sentiment-laden words in context
Test cases:      8658
Test cases run:  500
Fails (rate):    5 (1.0%)

Example fails:
0.2 0.0 0.8 This was a creepy flight.
----
0.1 0.0 0.9 This is an average aircraft.
----
0.0 0.0 1.0 It is an average staff.
----


neutral words in context
Test cases:      1716
Test cases run:  500
Fails (rate):    485 (97.0%)

Example fails:
0.0 0.0 1.0 This was an Australian seat.
----
0.1 0.0 0.9 This crew was Israeli.
----
0.0 0.0 1.0 I found this plane.
----


intensifiers
Test cases:      2000
Test cases run:  500
After filtering: 499 (99.8%)
Fails (rate):    14 (2.8%)

Example fails:
0.3 0.0 0.7 That was an aver

0.5 0.5 0.0 @VirginAmerica to start 5xweekly #A319 flights from to #Modesto @DallasLoveField #Austin on 28APR #avgeek

----
0.2 0.0 0.8 @JetBlue ETA for flight 802 into Orlando this morning
0.2 0.8 0.0 @JetBlue ETA for flight 802 into Yakima this morning
0.1 0.9 0.0 @JetBlue ETA for flight 802 into Springfield this morning

----
0.4 0.6 0.0 @united The Opal Dragon book The Dragon (ALI) has woven his murdering ways from the Philippines to Australia http://t.co/N2fvElcYgz
0.7 0.0 0.3 @united The Opal Dragon book The Dragon (ALI) has woven his murdering ways from the Mauritania to Australia http://t.co/N2fvElcYgz
0.7 0.0 0.3 @united The Opal Dragon book The Dragon (ALI) has woven his murdering ways from the Philippines to Senegal http://t.co/N2fvElcYgz

----


change numbers
Test cases:      1000
Test cases run:  500
Fails (rate):    15 (3.0%)

Example fails:
0.2 0.0 0.8 @USAirways strikes again--Late Flight crew #3/4 for the trip and maintenance #2/4.  Worth the extra $200/trip for less 

## QQP 

Transformers doesn't have a model pretrained on QQP, so we'll use one trained on MRPC for this demo

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")
model.to('cuda');
model.eval();

In [11]:
def batch_qqp(data, batch_size=128):
    ret = []
    for d in chunks(data, batch_size):
        t = tokenizer([a[0] for a in d], [a[1] for a in d], return_tensors='pt', padding=True).to('cuda')
        with torch.no_grad():
            logits = torch.softmax(model(**t)[0], dim=1).cpu().numpy()
        ret.append(logits)
    return np.vstack(ret)
    

In [12]:
from checklist.pred_wrapper import PredictorWrapper
# wrapped_pp returns a tuple with (predictions, softmax confidences)
wrapped_pp = PredictorWrapper.wrap_softmax(batch_qqp)

In [13]:
s0 = "The company HuggingFace is based in New York City"
s1 = "Apples are especially bad for your health"
s2 = "HuggingFace's headquarters are situated in Manhattan"
batch_qqp([(s0, s1), (s0, s2)])

array([[0.94038326, 0.05961675],
       [0.09536299, 0.904637  ]], dtype=float32)

In [14]:
suite_path = '../release_data/qqp/qqp_suite.pkl'
suite = TestSuite.from_file(suite_path)

In [15]:
suite.run(wrapped_pp, n=500)

Running Modifier: adj
Predicting 500 examples
Running different adjectives
Predicting 500 examples
Running Different animals
Predicting 500 examples
Running Irrelevant modifiers - animals
Predicting 500 examples
Running Irrelevant modifiers - people
Predicting 500 examples
Running Irrelevant preamble with different examples.
Predicting 500 examples
Running Preamble is relevant (different injuries)
Predicting 500 examples
Running How can I become more {synonym}?
Predicting 500 examples
Running (question, f(question)) where f(question) replaces synonyms?
Predicting 326 examples
Running Replace synonyms in real pairs
Predicting 684 examples
Running How can I become more X != How can I become less X
Predicting 500 examples
Running How can I become more X = How can I become less antonym(X)
Predicting 500 examples
Running add one typo
Predicting 1500 examples
Running contrations
Predicting 1427 examples
Running (q, paraphrase(q))
Predicting 18944 examples
Running Product of paraphrases(q1) *

In [16]:
suite.summary()

Vocabulary

Modifier: adj
Test cases:      1000
Test cases run:  500
Fails (rate):    499 (99.8%)

Example fails:
0.9 ('Is Timothy Cox an editor?', 'Is Timothy Cox an effective editor?')
----
0.9 ('Is Rebecca Stewart an accountant?', 'Is Rebecca Stewart an English accountant?')
----
0.9 ('Is Jeffrey Moore an administrator?', 'Is Jeffrey Moore an actual administrator?')
----


different adjectives
Test cases:      954
Test cases run:  500
Fails (rate):    438 (87.6%)

Example fails:
0.9 ('Is Andrea Diaz racist?', 'Is Andrea Diaz Jewish?')
----
0.9 ('Is Amanda Long racist?', 'Is Amanda Long an anarchist?')
----
0.9 ('Is Madison Brown English?', 'Is Madison Brown Armenian?')
----


Different animals
Test cases:      928
Test cases run:  500
Fails (rate):    500 (100.0%)

Example fails:
0.9 ('Can I feed my snail seeds?', 'Can I feed my turtle seeds?')
----
0.9 ('Can I feed my cat butter?', 'Can I feed my snail butter?')
----
0.9 ('Can I feed my lobster milk?', 'Can I feed my snail milk?')


## SQuAD

In [17]:
model = pipeline("question-answering", device=0)

In [18]:
suite_path = '../release_data/squad/squad_suite.pkl'
suite = TestSuite.from_file(suite_path)

Our suite has data as lists of (context, question) tuples. We'll adapt to this prediction format below:

In [19]:
def predconfs(context_question_pairs):
    preds = []
    confs = []
    for c, q in context_question_pairs:
        try:
            p = model(question=q, context=c, truncation=True, )
        except:
            print('Failed', q)
            preds.append(' ')
            confs.append(1)
        preds.append(p['answer'])
        confs.append(p['score'])
    return preds, np.array(confs)

In [20]:
# suite.tests['Question typo'].run(predconfs,n=100, overwrite=True)

In [21]:
suite.run(predconfs, n=100, overwrite=True)

Running A is COMP than B. Who is more / less COMP?
Predicting 200 examples
Running Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Predicting 1200 examples
Running size, shape, age, color
Predicting 400 examples
Running Profession vs nationality
Predicting 1000 examples
Running Animal vs Vehicle
Predicting 400 examples
Running Animal vs Vehicle v2
Predicting 400 examples
Running Synonyms
Predicting 400 examples
Running A is COMP than B. Who is antonym(COMP)? B
Predicting 400 examples
Running A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.
Predicting 1600 examples
Running Question typo
Predicting 200 examples


ERROR:transformers.tokenization_utils:We need to remove 115 to truncate the inputbut the first sequence has a length 21. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.


Failed What do the three richest people in the world posses more of than the lowest 48 nations together?


ERROR:transformers.tokenization_utils:We need to remove 116 to truncate the inputbut the first sequence has a length 22. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.


Failed What do the three richest people in hte world posses more of than the lowest 48 nations together?


...

Failed What did the Dutch health authorities regard as unnecessary in Geraets-Smits v Stichting Ziekenfonds?


ERROR:transformers.tokenization_utils:We need to remove 453 to truncate the inputbut the first sequence has a length 27. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.


Failed What did the Dutch health authorities regard as unnecessary in Geraets-Smits v Stichting Ziekenfonds?


ERROR:transformers.tokenization_utils:We need to remove 449 to truncate the inputbut the first sequence has a length 27. Please select another truncation strategy than TruncationStrategy.ONLY_FIRST, for instance 'longest_first' or 'only_second'.


Failed What did the Dutch health authorities regard as unnecessary in Geraets-Smits v Stichting Ziekenfonds?
Running There was a change in profession
Predicting 200 examples
Running Understanding before / after -> first / last.
Predicting 400 examples
Running Negation in context, may or may not be in question
Predicting 400 examples
Running Negation in question only.
Predicting 800 examples
Running M/F failure rates should be similar for different professions
Predicting 100 examples
Running Basic coref, he / she
Predicting 800 examples
Running Basic coref, his / her
Predicting 200 examples
Running Former / Latter
Predicting 400 examples
Running Agent / object distinction
Predicting 400 examples
Running Agent / object distinction with 3 agents
Predicting 1600 examples


In [23]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [24]:
suite.summary(format_example_fn=format_squad_with_context)

Vocabulary

A is COMP than B. Who is more / less COMP?
Test cases:      494
Test cases run:  100
Fails (rate):    100 (100.0%)

Example fails:
C: Timothy is cleaner than Kelly.
Q: Who is less clean?
A: Kelly
P: Timothy


----
C: David is older than Samantha.
Q: Who is less old?
A: Samantha
P: David


----
C: Michael is faster than Jose.
Q: Who is less fast?
A: Jose
P: Michael


----


Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Test cases:      497
Test cases run:  100
Fails (rate):    100 (100.0%)

Example fails:
C: Maria is open about the project. Michelle is a little open about the project.
Q: Who is least open about the project?
A: Michelle
P: Maria

C: Michelle is a little open about the project. Maria is open about the project.
Q: Who is least open about the project?
A: Michelle
P: Maria

C: Maria is very open about the project. Michelle is a little open about the project.
Q: Who is least open about the project?
A: Michelle
P: Maria


----
C: Victor